In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sn 
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from termcolor import colored
from sklearn.metrics import log_loss


In [ ]:
val_list = ['val_1.csv', 'val_2.csv', 'val_3.csv', 'val_4.csv', 'val_5.csv']
train_list = ['train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv', 'train_5.csv']
test_list = ['test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv', 'test_5.csv']
count = 1 
test_acc = []
val_acc = []

# print(colored("\nInitialization Complete", "yellow"))

for fold in range(5):

    if fold == 1 or 2: 
        pen = "l1"
        c = 0.1
    elif fold == 3 or 5:
        pen = "l1"
        c = 0.3
    elif fold == 4:
        pen = "l1"
        c = 0.5

    print("\n\n")
    print(colored("\tREADING IN DATA", "red"))
    print("\tFold", str(count))
    
    #read in validaiton data
    X_val= pd.read_csv(val_list[fold], header = 0, delimiter = ',')
    X_val.loc[X_val["category"] == "luminalA", "class"] = "0"
    X_val.loc[X_val["category"] == "luminalB", "class"] = "1"
    X_val.loc[X_val["category"] == "tripleNegative", "class"] = "2"
    X_val.loc[X_val["category"] == "her2Enriched", "class"] = "3"
    X_val = X_val.drop(['Unnamed: 0', 'bcr_patient_barcode', 'category'], axis = 1)
    Y_val = X_val['class']

    #read in training data 
    X_train = pd.read_csv(train_list[fold], header = 0, delimiter = ',')
    X_train.loc[X_train["category"] == "luminalA", "class"] = "0"
    X_train.loc[X_train["category"] == "luminalB", "class"] = "1"
    X_train.loc[X_train["category"] == "tripleNegative", "class"] = "2"
    X_train.loc[X_train["category"] == "her2Enriched", "class"] = "3"
    X_train = X_train.drop(['Unnamed: 0', 'bcr_patient_barcode', 'category'], axis = 1)
    Y_train = X_train['class']


    #fit model 
    print(colored("\tFITTING MODEL", "red"))
    #Create LR object and fit model on training data 

    if pen == "l1":
        lr = LogisticRegression(multi_class = "ovr", solver = "liblinear", max_iter = 10000, penalty = pen, C = c)
    else: 
        lr = LogisticRegression(multi_class = "ovr", solver = "lbfgs", n_jobs = 4, max_iter = 10000, penalty = pen, C = c)
    lr.fit(X_train, Y_train)
    # print("\tNum iterations", lr.n_iter_)
    print(colored("\tTraining Accuracy", "green"))
    print("\t\t", lr.score(X_train, Y_train))


    ##Calculate validation accuracy 
    print(colored("\tMODEL VALIDATION ACCURACY", "red"))
    #Model Accuracy 
    Y_predicted = lr.predict(X_val)
    print(colored("\tValidation Accuracy", "green"))
    # print(Y_predicted)
    print("\t\t", lr.score(X_val, Y_val))
    val_acc.append(lr.score(X_val, Y_val))
    # cm = confusion_matrix(Y_val, Y_predicted)


    #print confusion matrix 
    # print(colored("\tCREATING CONFUSION MATRIX", "red"))
    # # import seaborn as sn 
    # plt.figure(figsize = (10,7))
    # sn.heatmap(cm, annot = True, xticklabels = ['Luminal A', 'Luminal B', 'Triple Negative', 'HER2-enriched'], yticklabels = ['Luminal A', 'Luminal B', 'Triple Negative', 'HER2-enriched'], annot_kws={"fontsize":18})
    # plt.xlabel('Predicted')
    # plt.ylabel('Truth')
    # plt.savefig("fold_"+ str(count)+"_"+str(pen)+"_cm.png")



    ###----------------GRID SEARCH------------------##

    # print(colored("GRID SEARCH" , "yellow"))
    # count = 1
    # for fold in range(5):
    #     #    read in validaiton data
    #     print(colored("\t Fold "+str(count), "blue"))
    #     X_val= pd.read_csv( val_list[fold], header = 0, delimiter = ',')
    #     X_val.loc[X_val["category"] == "luminalA", "class"] = "0"
    #     X_val.loc[X_val["category"] == "luminalB", "class"] = "1"
    #     X_val.loc[X_val["category"] == "tripleNegative", "class"] = "2"
    #     X_val.loc[X_val["category"] == "her2Enriched", "class"] = "3"
    #     X_val = X_val.drop(['Unnamed: 0', 'bcr_patient_barcode', 'category'], axis = 1)
    #     Y_val = X_val['class']

    #     logistic = LogisticRegression(multi_class = "ovr", max_iter = 10000)
    #     penalty = ["l2", "l1"]
    #     C = np.logspace(0.001, 0.01, 0.1, 0.3, 0.5, 1.0, 1.3, 1.5, 3.0, 5.0)
    #     hyperparameters = dict(C = C, penalty = penalty)

    #     clf = GridSearchCV(logistic, hyperparameters, cv = 5)

    #     best_model = clf.fit(X_val, Y_val)

    #     print(colored("\tOutput Fold "+ str(count), "green"))
    #     print('\t\tBest Penalty: ', best_model.best_estimator_.get_params()['penalty'])
    #     print('\t\tBest C: ', best_model.best_estimator_.get_params()['C'])

    ###----------------GRID SEARCH------------------##



    #calculate TESTING acurracy
    print(colored("\tTESTING ACCURACY SCORE FOR THIS FOLD", "red"))
    #testing accuracy score 
    ##read in testing data 
    X_test = pd.read_csv(test_list[fold], header = 0, delimiter = ',')
    X_test.loc[X_test["category"] == "luminalA", "class"] = "0"
    X_test.loc[X_test["category"] == "luminalB", "class"] = "1"
    X_test.loc[X_test["category"] == "tripleNegative", "class"] = "2"
    X_test.loc[X_test["category"] == "her2Enriched", "class"] = "3"
    X_test = X_test.drop(['Unnamed: 0', 'bcr_patient_barcode', 'category'], axis = 1)
    Y_test = X_test['class']

    test_acc.append(lr.score(X_test, Y_test))
    print(colored("\t\tTesting accuracy - fold "+ str(count)+": "+ str(lr.score(X_test, Y_test)), "green"))
    Y_predicted = lr.predict(X_test)
    from sklearn.metrics import classification_report
    print(classification_report(Y_test, Y_predicted, target_names = ['0', '1', '2', '3']))

    # print("\t\tSaving Test Accuracy Confusion Matrix...")
    # cm_test = confusion_matrix(X_test, Y_test)
    # plt.figure(figsize = (10,7))
    # sn.heatmap(cm_test, annot = True, xticklabels = ['Luminal A', 'Luminal B', 'Triple Negative', 'HER2-enriched'], yticklabels = ['Luminal A', 'Luminal B', 'Triple Negative', 'HER2-enriched'], annot_kws={"fontsize":18}) #annot_kws={"fontsize":18}
    # plt.xticks(rotation = 90)
    # plt.yticks(rotation = 0)
    # plt.xlabel('Predicted')
    # plt.ylabel('Truth')
    # plt.savefig("fold_"+str(count)+"_cm_TEST.png")
    # print("\t\t\tDone")

    count +=1

    #end inner for 


print(colored("\n\tCV Accuracy_ = {}\n".format(sum(test_acc) / 5), "magenta"))
print("")

#end outer for 

print(colored("\n---------------------- D O N E --------------------\n\n", "green"))


